Filter the Blast hits for >50% coverage and >30% identity. count the residues at each position of the BoNT/F alignment. Write a function to generate a list of residues with conservation above a specified cutoff.

In [ ]:
from Bio import SeqIO
import pandas as pd

Import the fasta file for your target protein

In [ ]:
target_fasta = list(SeqIO.parse('BoNT_X.fasta','fasta'))

In [ ]:
name = target_fasta[0].id

Import the data table output (.tsv) of the initial blast search

In [ ]:
df = pd.read_csv('BoNTX_out_-4.tsv',sep='\t',header=None)

In [ ]:
column_titles = ['query','id','match','length','mismatch','gapopen','qstart','qend','sstart','send','Eval','bitscore']

In [ ]:
df.columns = column_titles

In [ ]:
prot_length = len(target_fasta[0].seq)

In [ ]:
filtered_ids = []
for index, row in df.iterrows():
    if row['match'] > 30:
        coverage = (prot_length)/2
        if row['length'] > coverage:
            filtered_ids.append(row['id'])
        

analyzed the filtered sequences. Import the .fasta file of Blast hit sequences:

In [ ]:
blast_fasta = list(SeqIO.parse('X_hits_seqs.fasta','fasta'))

In [ ]:
filtered_seqs = {}
entries = []
entries.append(target_fasta[0]) #add BoNT/E to the top

for record in blast_fasta:
    if record.id in filtered_ids:
        filtered_seqs[record.id] = record.seq
        entries.append(record)
        
SeqIO.write(entries, "filtered_hits.fasta", "fasta")
    

Make MSA from the fasta file. The next steps will make a multiple sequence aligment with clustal omega. If you don't have clustal omega installed you can make the multiple alignment of the filtered_hits.fasta using Geneious, move the file to this directory, then skip ahead to the next note

In [ ]:
from Bio.Align.Applications import ClustalOmegaCommandline

In [ ]:
from Bio import AlignIO


In [ ]:
clustalomega_cline = ClustalOmegaCommandline(infile='filtered_hits.fasta', outfile='filtered_aligned.fasta',force=True)
clustalomega_cline()


Skip to this step if you aligned in Geneious and import the file you generated

In [ ]:
# Parse the resulting alignment file
alignment = AlignIO.read('filtered_aligned.fasta', "fasta")

Add up the counts of each non-gap AA at each position 

In [ ]:
positions_dict = {}
for pos in range(alignment.get_alignment_length()):
    dict = {}
    for i in range(len(alignment)):
        if alignment[i][pos] != '-':
            if alignment[i][pos] not in dict:
                dict[alignment[i][pos]] = 1
            else:
                dict[alignment[i][pos]] += 1
    
    positions_dict[pos] = dict
    

Determine the fraction consensus and most abundant AA (not including gaps in count)

In [ ]:
consensus = []
consensus_AA = []
for i in range(len(positions_dict)):
    key_with_highest_value = max(positions_dict[i], key=lambda key: positions_dict[i][key])
    consensus_AA.append(key_with_highest_value)
    
    total = 0
    for value in positions_dict[i].values():
        total += value
    consensus.append(max(positions_dict[i].values())/total)

copy in the distance-constrained residue list that you calculated in pyrosetta (colab)

In [ ]:
PDB_fixed_res = [23,
 24,
 25,
 26,
 27,
 28,
 34,
 35,
 36,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 60,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 124,
 132,
 133,
 134,
 135,
 136,
 137,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 172,
 172,
 172,
 173,
 174,
 175,
 176,
 177,
 179,
 180,
 181,
 182,
 183,
 184,
 185,
 186,
 187,
 192,
 193,
 223,
 224,
 225,
 226,
 227,
 228,
 229,
 230,
 231,
 232,
 233,
 234,
 235,
 236,
 238,
 239,
 240,
 241,
 242,
 243,
 244,
 244,
 245,
 246,
 247,
 248,
 249,
 250,
 252,
 254,
 257,
 258,
 259,
 260,
 261,
 262,
 263,
 264,
 265,
 266,
 267,
 268,
 269,
 270,
 271,
 274,
 275,
 348,
 360,
 361,
 362,
 363,
 364,
 365,
 366,
 367,
 368,
 422,
 29,
 30,
 31,
 32,
 33]


Define the function to choose residues to fix based on selected conservation cutoffs and combine with the distance constraint list

In [ ]:
def get_res(cutoffList,distance_cutoff):
    output_dict = {}

    #choose conserved residues to fix based on cutoff
    for cutoff in cutoffList:
        fixed_cons = []
        x = 0
        sequence = str(alignment[0].seq)
        for i,res in enumerate(sequence):
            if res != '-':
                x += 1
                if res == consensus_AA[i]:
                    if consensus[i] >= cutoff:
                        fixed_cons.append(x)
    
        frac = len(fixed_cons)/len(target_fasta[0].seq)
    
        #make list of the fixed residues from distance or conservation requirement
        all_fixed_residues = []
        for x in fixed_cons:
            all_fixed_residues.append(x)
        for y in PDB_fixed_res:
            if y not in all_fixed_residues:
                all_fixed_residues.append(y)
                
        #for z in man_fixed_res:
         #   if z not in all_fixed_residues:
          #      all_fixed_residues.append(z)
                
        all_fixed_residues.sort()
        
        frac_all_fixed = len(all_fixed_residues)/len(target_fasta[0].seq)

        all_fixed_res_str = ''
        for i in all_fixed_residues:
            all_fixed_res_str= all_fixed_res_str + str(i) + ' '
    
        #pymol commands to select residues
        pymol_command_conserve = 'select conserve, chain A and resi '
        for x in fixed_cons:
          pymol_command_conserve = pymol_command_conserve + str(x) + '+'
    
        pymol_command_bind = 'select bind, chain A and resi '
        for x in PDB_fixed_res:
          pymol_command_bind = pymol_command_bind + str(x) + '+'
    
        #pymol_command_man = 'select manual, chain A and resi '
        #for x in man_fixed_res:
          #pymol_command_man = pymol_command_man + str(x) + '+'

        output_dict[cutoff] = [all_fixed_res_str,all_fixed_residues, fixed_cons,frac, frac_all_fixed, pymol_command_conserve, pymol_command_bind]

    df = pd.DataFrame.from_dict(output_dict)
    cols = ['MPNN fix string','all fixed','fixed by conservation','fraction fixed by conservation','fraction all res fixed','pymol conserved','pymol bind']
    df=df.T
    df.columns=cols
    df.to_csv(name+'_'+str(distance_cutoff)+'_constraints.csv')
        
    

Run the function with your desired inputs. The first argument is a list of your conservation cutoffs (30% and 60% in the example) and the second argument is the distance cutoff you used in your distance analysis

In [ ]:
get_res([.3, .6],14)

A csv file should have been generated with the list of fixed residues for each conservation cutoff and the particular distance cutoff. To generate constraints for different distance cutoffs, rerun this script with the distance-constrained residue lists you generated for those cutoffs.